In [1]:
import tensorflow as tf
from tensorflow import keras
import kerastuner
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime
import dill
import tqdm
from queue import PriorityQueue

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
from random import shuffle
from copy import copy

class TreeTools:
    def __init__(self):
        #memoization for _count_nodes functions
        self._count_nodes_dict = {}
                
    def _get_subtrees(self, tree):
        yield tree
        for subtree in tree:
            if type(subtree) == list:
                for x in self._get_subtrees(subtree):
                    yield x

    # Returns pairs of paths and leaves of a tree
    def _get_leaves_paths(self, tree):
        for i, subtree in enumerate(tree):
            if type(subtree) == list:
                for path, value in self._get_leaves_paths(subtree):
                    yield [i] + path, value
            else:
                yield [i], subtree
    
    # Returns the number of nodes in a tree (not including root)
    def _count_nodes(self, tree):
        if id(tree) in self._count_nodes_dict:
            return self._count_nodes_dict[id(tree)]
        size = 0
        for node in tree:
            if type(node) == list:
                size += 1 + self._count_nodes(node)
        self._count_nodes_dict[id(self._count_nodes_dict)] = size
        return size


    # Returns all the nodes in a path
    def _get_nodes(self, tree, path):
        next_node = 0
        nodes = []
        for decision in path:
            nodes.append(next_node)
            next_node += 1 + self._count_nodes(tree[:decision])
            tree = tree[decision]
        return nodes

    def _value_to_path_nodes_dict(self, tree):
        value_to_path_nodes_dict = {}
        for path, value in tqdm.tqdm(self._get_leaves_paths(tree)):
            nodes = self._get_nodes(tree, path)
            value_to_path_nodes_dict[value] = path, nodes
        return value_to_path_nodes_dict


# turns a list to a binary tree
def random_binary_full_tree(outputs):
    outputs = copy(outputs)
    shuffle(outputs)

    while len(outputs) > 2:
        temp_outputs = []
        for i in range(0, len(outputs), 2):
            if len(outputs) - (i+1) > 0:
                temp_outputs.append([outputs[i], outputs[i+1]])
            else:
                temp_outputs.append(outputs[i])
        outputs = temp_outputs
    return outputs

In [3]:
tree = random_binary_full_tree(list(range(10)))
print('Our tree:',tree)

tree_tools = TreeTools()    

print('All subtrees:')
for subtree in tree_tools._get_subtrees(tree):
    print('\t {} (Len : {})'.format(subtree, len(subtree)))

print('All paths and leaves:')
for subtree in tree_tools._get_leaves_paths(tree):
    print('\t',subtree)
    
print('Number of nodes in the tree:',tree_tools._count_nodes(tree))

print('all nodes in path [0, 0, 0, 0]:')
for nodes in tree_tools._get_nodes(tree, [0, 0, 0, 0]):
    print('\t',nodes)

print('all nodes in path [1, 0]:')
for nodes in tree_tools._get_nodes(tree, [1, 0]):
    print('\t',nodes)

Our tree: [[[[4, 8], [5, 6]], [[0, 2], [9, 7]]], [1, 3]]
All subtrees:
	 [[[[4, 8], [5, 6]], [[0, 2], [9, 7]]], [1, 3]] (Len : 2)
	 [[[4, 8], [5, 6]], [[0, 2], [9, 7]]] (Len : 2)
	 [[4, 8], [5, 6]] (Len : 2)
	 [4, 8] (Len : 2)
	 [5, 6] (Len : 2)
	 [[0, 2], [9, 7]] (Len : 2)
	 [0, 2] (Len : 2)
	 [9, 7] (Len : 2)
	 [1, 3] (Len : 2)
All paths and leaves:
	 ([0, 0, 0, 0], 4)
	 ([0, 0, 0, 1], 8)
	 ([0, 0, 1, 0], 5)
	 ([0, 0, 1, 1], 6)
	 ([0, 1, 0, 0], 0)
	 ([0, 1, 0, 1], 2)
	 ([0, 1, 1, 0], 9)
	 ([0, 1, 1, 1], 7)
	 ([1, 0], 1)
	 ([1, 1], 3)
Number of nodes in the tree: 8
all nodes in path [0, 0, 0, 0]:
	 0
	 1
	 2
	 3
all nodes in path [1, 0]:
	 0
	 8


In [4]:
tree_tools._value_to_path_and_nodes_dict(tree)

AttributeError: 'TreeTools' object has no attribute 'value_to_path_and_nodes_dict'

In [4]:
class hier_softmax:
    def __init__(self, tree, contex_size, model):
        self._tree_tools = TreeTools()
        self.str2weight = {}
        #create a weight matrix and bias vector for each node in the tree
        for i, subtree in enumerate(self._tree_tools._get_subtrees(tree)):
            self.str2weight["softmax_node_"+str(i)+"_w"] = model.add_parameters((len(subtree), contex_size))
            self.str2weight["softmax_node_" + str(i) + "_b"] = model.add_parameters(len(subtree))
        
        #create a dictionary from each value to its path
        value_to_path_and_nodes_dict = {}
        for path, value in self._tree_tools._get_leaves_paths(tree):
            nodes = self._tree_tools._get_nodes(tree, path)
            value_to_path_and_nodes_dict[data.char2int[value]] = path, nodes
        self.value_to_path_and_nodes_dict = value_to_path_and_nodes_dict
        self.model = model
        self.tree = tree
    
    #get the loss on a given value (for training)
    def get_loss(self, context, value):
        loss = []
        path, nodes = self.value_to_path_and_nodes_dict[value]
        for p, n in zip(path, nodes):
            w = dy.parameter(self.str2weight["softmax_node_"+str(n)+"_w"])
            b = dy.parameter(self.str2weight["softmax_node_" + str(n) + "_b"])
            probs = tf.nn.softmax(w*context+b)
            #loss.append(-tf.math.log(dy.pick(probs, p)))
            print(probs)
            print(p)
        #return dy.esum(loss)

    #get the most likely
    def generate(self, context):
        best_value = None
        best_loss = float(100000)
        for value in self.value_to_path_and_nodes_dict:
            loss = self.get_loss(context, value)
            if loss < best_loss:
                best_loss = loss
                best_value = value
        return best_value

Huffman Encoding 

# Main function implementing huffman coding
def huffman_code_tree(node, path, left=True):
    if type(node) is not Node:
        return {node: path}
    (l, r) = node.children()
    d = dict()

    l_path = copy(path)
    l_path.append(0)
    r_path = copy(path)
    r_path.append(1)

    d.update(huffman_code_tree(l, l_path, True))
    d.update(huffman_code_tree(r, r_path, False))
    return d

nodes = list(freq.items())

while len(nodes) > 1:
    (key1, c1) = nodes[-1]
    (key2, c2) = nodes[-2]
    nodes = nodes[:-2]
    node = Node(key1, key2)
    nodes.append((node, c1 + c2))

    nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

huffmanCode = huffman_code_tree(nodes[0][0], [])
huffmanCode

Huffman Binary Tree as Node (not as list)

class Node(object):
    node_id = 0
    def __init__(self, symbol, freq, left=None, right=None):
        self.left = left
        self.right = right
        self.symbol = symbol
        self.freq = freq

        if self.symbol == None:
            self.node_id = Node.node_id
            Node.node_id += 1

    def children(self):
        return (self.left, self.right)

    def __lt__(self, target):
        if type(target) == Node:
            return self.freq < target.freq
        else:
            return self.freq < target
    
    def __gt__(self, target):
        return not self.__lt__(target)

nodes = list(freq.items())
q = PriorityQueue()

for node in nodes:
    q.put(Node(node[0], node[1]))

while q.qsize() > 1:
    node_1 = q.get()
    node_2 = q.get()
    print(node_1.symbol, node_1.freq)
    print(node_2.symbol, node_2.freq)
    node = Node(None, node_1.freq + node_2.freq, node_1, node_2)
    q.put(node)

def traverse(tree):
    l, r = tree.children()
    if l == None or r == None:
        print(tree.symbol)
        return
    #print(tree.freq)    
    
    traverse(l)
    traverse(r)

traverse(tree)

In [3]:
class Node(object):
    '''
    Wrapping List with Node class Since pythn PriorityQueue cannot recognize nested items in its input tuple.
    '''
    def __init__(self, symbol, freq, symbol_2=None):
        if symbol_2:
            self.symbol = [symbol, symbol_2]
        else:
            self.symbol = symbol
        self.freq = freq

    def __lt__(self, target):
        if type(target) == Node:
            return self.freq < target.freq
        else:
            return self.freq < target
    
    def __gt__(self, target):
        return not self.__lt__(target)

def create_huffman_tree(input_dict):
    '''
    input : dictionary with {category:freq} pairs
    NOTE : input dict must be ASCENDING order
    '''
    q = PriorityQueue()
    nodes = list(input_dict.items())

    for node in nodes:
        q.put(Node(node[0], node[1]))

    while q.qsize() > 1:
        node_1 = q.get()
        node_2 = q.get()
        #print(node_1.symbol, node_1.freq)
        #print(node_2.symbol, node_2.freq)
        node = Node(node_1.symbol, node_1.freq + node_2.freq, node_2.symbol)
        q.put(node)
    return q.get().symbol

In [4]:
train_set = np.genfromtxt("data/{}_train_set_2.csv".format("SEG_Wavenet"), delimiter="\n", dtype=np.int64)
val_set = np.genfromtxt("data/{}_val_set_2.csv".format("SEG_Wavenet"), delimiter="\n", dtype=np.int64)

In [5]:
dataset = np.r_[train_set, val_set]
dataset

array([46144, 52215, 23250, ...,   889,   246,  1385], dtype=int64)

In [6]:
dataset_freq = pd.Series(dataset).value_counts(ascending=True).to_dict()

In [7]:
pd.Series(dataset).value_counts()

1        26424
2         6295
3         1849
4         1848
5         1207
         ...  
53853        1
33367        1
35414        1
37461        1
36397        1
Length: 55277, dtype: int64

In [8]:
tree = create_huffman_tree(dataset_freq)

In [9]:
tree_tools = TreeTools()    
'''
print('All subtrees:')      # Num of All subtrees : 55276
for subtree in tree_tools._get_subtrees(tree):
    print('\t {} (Len : {})'.format(subtree, len(subtree)))

print('All paths and leaves:')
for subtree in tree_tools._get_leaves_paths(tree):
    print('\t',subtree)
'''
num_nodes = tree_tools._count_nodes(tree)
print('Number of nodes in the tree:', num_nodes) # Num of All nodes (except root): 55275

print('all nodes in path [0, 0, 0, 0]:')
for nodes in tree_tools._get_nodes(tree, [1, 1, 0, 0]):
    print('\t',nodes)

print('all nodes in path [1, 0]:')
for nodes in tree_tools._get_nodes(tree, [1, 0]):
    print('\t',nodes)

Number of nodes in the tree: 55275
all nodes in path [0, 0, 0, 0]:
	 0
	 17143
	 19783
	 19784
all nodes in path [1, 0]:
	 0
	 17143


In [14]:
mapping_dict = tree_tools._value_to_path_nodes_dict(tree)

55277it [15:20, 60.08it/s]


import json

with open("static/mappding_dict.json", "w") as j:
    json.dump(mapping_dict, j, indent=4)

In [77]:
class HierachicalSoftmax(keras.Model):
    def __init__(self, mapping_dict):
        super().__init__()

        self.mapping_dict = mapping_dict
        self.decision_function = {1:1, 0:-1}

    def build(self, input_shape):
        self.initializer = keras.initializers.GlorotNormal()
        self.softmax_weights = tf.Variable(self.initializer(shape=(len(self.mapping_dict)-1, input_shape[-1], 1)))

    def call(self, x):
        total_loss =[]
        for category, (path, nodes) in  self.mapping_dict.items()[:11]:
            loss = []
            for p, n in zip(path, nodes):
                w = self.softmax_weights[n]
                d = self.decision_function[p]
                sigma = tf.nn.softmax(w*x)
                loss.append(tf.tensordot(sigma, d))
            total_loss.append(tf.math.reduce_prod(loss))
        return total_loss

    def train_step(self, data):
        loss = []
        for p, n in zip(path, nodes):
            raw_prob = tf.nn.softmax(x * self.softmax_weights[n])
            loss.append(tf.tensordot(raw_prob, self.decision_function[p]))
        return tf.reduce_prod(loss, axis=-1)

In [65]:
mapping_dict[0]

KeyError: 0

In [73]:
len(mapping_dict)

55277

In [66]:
mapping_dict[1]

(ListWrapper([1, 0, 0]), ListWrapper([0, 17143, 17144]))

In [76]:
for category, (path, nodes) in mapping_dict.items():
    print(category, path, nodes)
    break

20456 ListWrapper([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) ListWrapper([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])


In [67]:
tf.math.multiply(tf.constant([1, 2, 1]), tf.constant([2, 3, 4]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 6, 4])>

In [68]:
tf.reduce_prod(tf.constant([1, 2, 3]))

<tf.Tensor: shape=(), dtype=int32, numpy=6>

In [80]:
x = tf.random.uniform((1, 1, 30))
x

<tf.Tensor: shape=(1, 1, 30), dtype=float32, numpy=
array([[[0.18911195, 0.37342346, 0.9745028 , 0.66212094, 0.879431  ,
         0.7414323 , 0.5117035 , 0.6403229 , 0.67704856, 0.3146212 ,
         0.8127583 , 0.05822527, 0.18394876, 0.30106592, 0.4722501 ,
         0.98014295, 0.6283145 , 0.16613781, 0.6566857 , 0.66950905,
         0.24273705, 0.60346067, 0.5234035 , 0.5597247 , 0.9225699 ,
         0.3113091 , 0.69289136, 0.24133515, 0.9949957 , 0.05192733]]],
      dtype=float32)>

In [81]:
hsm = HierachicalSoftmax(mapping_dict)

In [72]:
hsm(x)

TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

In [82]:
initializer = keras.initializers.GlorotNormal()
softmax_weights = tf.Variable(initializer(shape=(len(mapping_dict)-1, x.shape[-1], 1)))

In [88]:
decision_function = {1:1, 0:-1}

def call(x):
    total_loss =[]
    for category, (path, nodes) in list(mapping_dict.items())[:11]:
        loss = []
        for p, n in zip(path, nodes):
            w = softmax_weights[n]
            d = decision_function[p]
            sigma = tf.nn.softmax(w*x)
            loss.append(tf.tensordot(sigma, d, axes=))
        total_loss.append(tf.math.reduce_prod(loss))
    return total_loss

In [89]:
call(x)

TypeError: tensordot() missing 1 required positional argument: 'axes'